In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Prompt Optimizer Notebook SDK (Preview) - Tool usage

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_sdk_custom_metric.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fprompt_optimizer%2Fvertex_ai_prompt_optimizer_sdk_custom_metric.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_sdk_custom_metric.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/prompt_optimizer/vertex_ai_prompt_optimizer_sdk_custom_metric.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>
    

| | | |
|-|-|-|
| Author | [Ivan Nardini](https://github.com/inardini)

##  I. Overview

When developing Generative AI (Gen AI) applications, prompt engineering poses challenges due to its time-consuming and error-prone nature. Significant effort is involved when crafting and inputting prompts to achieve successful task completion. With the frequent release of foundational models, you face the added burden of migrating working prompts from one model version to another.

Vertex AI prompt optimizer alleviates these challenges by providing an intelligent prompt optimization tool. With this tool you can both translate and optimize system instructions in the prompts and best demonstrations (examples) for prompt templates, which lets you shape LLM responses from any source model to a target Google model.


### Objective

This notebook demonstrates how to leverage Vertex AI prompt optimizer to optimize for tool usage with a Gemini model. The goal is to use Vertex AI prompt optimizer to find a new prompt template which improves the model's ability to predict valid tool (function) calls given user's request.

This tutorial uses the following Google Cloud services and resources:

- Generative AI on Vertex AI
- Vertex AI prompt optimizer
- Vertex AI Gen AI evaluation
- Vertex AI Custom job

The steps performed include:

1. Define the prompt template you want to optimize.
2. Prepare the prompt optimization dataset.
3. Configure tool function settings and validate them.
4. Set optimization mode and steps.
5. Run the automatic prompt optimization job.
6. Collect the best prompt template and eval metric.
7. Validate the best prompt template.

### Dataset

The dataset is a question-answering dataset generated by a simple AI financial assistant that provides information about the top 25 Tech companies.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## II. Before you start

### Install Vertex AI SDK for Python and other required packages


In [ ]:
%pip install --upgrade --quiet 'google-cloud-aiplatform[evaluation]'
%pip install --upgrade --quiet 'plotly' 'asyncio' 'tqdm' 'tenacity' 'etils' 'importlib_resources' 'fsspec' 'gcsfs' 'nbformat>=4.2.0' 'jsonschema'

In [ ]:
! mkdir -p ./tutorial/utils && wget https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/prompt_optimizer/vapo_lib.py -P ./tutorial/utils

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
# import sys

# if "google.colab" in sys.modules:
#     from google.colab import auth

#     auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the following APIs](https://console.cloud.google.com/flows/enableapi?apiid=cloudresourcemanager.googleapis.com,aiplatform.googleapis.com,cloudfunctions.googleapis.com,run.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

#### Set your project ID and project number

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format="get(projectNumber)"[0]
PROJECT_NUMBER = PROJECT_NUMBER[0]

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_NAME = "your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

#### Service Account and permissions

Vertex AI Prompt optimizer requires a service account with the following permissions:

-   `Vertex AI User` to call Vertex LLM API
-   `Storage Object Admin` to read and write to your GCS bucket.

[Check out the documentation](https://cloud.google.com/iam/docs/manage-access-service-accounts#iam-view-access-sa-gcloud) to learn how to grant those permissions to a single service account.


> If you run following commands using Vertex AI Workbench, please directly run in the terminal.


In [ ]:
SERVICE_ACCOUNT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

In [ ]:
for role in ['aiplatform.user', 'storage.objectAdmin']:

    ! gcloud projects add-iam-policy-binding {PROJECT_ID} \
      --member=serviceAccount:{SERVICE_ACCOUNT} \
      --role=roles/{role} --condition=None

### Set workspace

Set a workspace to store prompt optimization results on Cloud Storage bucket.

In [ ]:
from etils import epath

WORKSPACE_URI = epath.Path(BUCKET_URI) / "optimization"
INPUT_DATA_URI = epath.Path(WORKSPACE_URI) / "data"

WORKSPACE_URI.mkdir(parents=True, exist_ok=True)
INPUT_DATA_URI.mkdir(parents=True, exist_ok=True)

### Import libraries

In [ ]:
# Tutorial
from argparse import Namespace
from typing import Any, Dict, List, Union, Optional, Callable
import json
import jsonschema

# General
import logging
from pprint import pprint
import warnings

from IPython.display import HTML, display
from google.cloud import aiplatform
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from tutorial.utils import vapo_lib
from tqdm.asyncio import tqdm_asyncio

### Libraries settings

In [ ]:
warnings.filterwarnings("ignore")
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)

### Define constants

In [ ]:
INPUT_DATA_FILE_URI = "gs://github-repo/prompts/prompt_optimizer/qa_tool_calls_dataset.jsonl"

INPUT_OPTIMIZATION_DATA_URI = epath.Path(WORKSPACE_URI) / "prompt_optimization_data"
INPUT_OPTIMIZATION_DATA_FILE_URI = str(
    INPUT_DATA_URI / "prompt_optimization_dataset.jsonl"
)
OUTPUT_OPTIMIZATION_DATA_URI = epath.Path(WORKSPACE_URI) / "optimization_jobs"
APD_CONTAINER_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/apd:preview_v1_0"
)
CONFIG_FILE_URI = str(WORKSPACE_URI / "config" / "config.json")

In [ ]:
INPUT_DATA_FILE_URI = "gs://vapo-tool-opt/optimization/data/qa_tool_calls_dataset.jsonl"

### Set helpers

In [ ]:
def get_company_information_api(content: Dict[str, Any]) -> str:
  "A function to simulate an API call to collect company information."

  company_overviews = {
 'AAPL': 'Apple maintains a robust financial position with substantial cash reserves and consistent profitability, fueled by its strong brand and loyal customer base. However, growth is slowing and the company faces competition.',
 'ADBE': 'Adobe financials are robust, driven by its successful transition to a subscription-based model for its creative and document cloud software.  Profitability and revenue growth are strong.',
 'AMD': 'AMD exhibits strong financial performance, gaining market share in the CPU and GPU markets.  Revenue growth and profitability are healthy, driven by strong product offerings.',
 'AMZN': 'Amazon financials are mixed, with its e-commerce business facing margin pressure while its cloud computing division (AWS) delivers strong profitability and growth. Its overall revenue remains high but profitability is a concern.',
 'ASML': 'ASML boasts a strong financial position due to its monopoly in the extreme ultraviolet lithography market, essential for advanced semiconductor manufacturing.  High profitability and growth are key strengths.',
 'AVGO': 'Broadcom maintains healthy financials, driven by its semiconductor and infrastructure software solutions. Acquisitions have played a role in its growth strategy, with consistent profitability and cash flow.',
 'BABA': 'Alibaba financials are substantial but facing challenges from regulatory scrutiny in China and increased competition.  E-commerce revenue remains strong but growth is slowing.',
 'BKNG': 'Booking Holdings financials are closely tied to the travel industry.  Revenue growth is recovering post-pandemic but profitability can fluctuate based on global travel trends.',
 'CRM': 'Salesforce shows robust revenue growth from its cloud-based CRM solutions.  Profitability is improving but competition remains strong.',
 'CSCO': 'Cisco financials show moderate growth, transitioning from hardware to software and services.  Profitability is stable but the company faces competition in the networking market.',
 'GOOGL': 'Alphabet exhibits strong financials driven by advertising revenue, though facing regulatory scrutiny.  Diversification into other ventures provides growth opportunities but profitability varies.',
 'IBM': 'IBM financials are in a state of transformation, shifting focus to hybrid cloud and AI.  Revenue growth is modest, with profitability impacted by legacy businesses.',
 'INTU': 'Intuit showcases healthy financials, benefiting from its strong position in tax and financial management software.  Revenue growth and profitability are consistent, fueled by recurring subscription revenue.',
 'META': 'Meta Platforms financial performance is tied closely to advertising revenue, facing headwinds from competition and changing privacy regulations.  Investments in the metaverse represent a long-term, high-risk bet.',
 'MSFT': 'Microsoft demonstrates healthy financials, benefiting from diversified revenue streams including cloud computing (Azure), software, and hardware.  The company exhibits consistent growth and profitability.',
 'NFLX': 'Netflix exhibits strong revenue but faces challenges in maintaining subscriber growth and managing content costs. Profitability varies, and competition in the streaming market is intense.',
 'NOW': 'ServiceNow demonstrates strong financials, fueled by its cloud-based workflow automation platform.  Revenue growth and profitability are high, reflecting increased enterprise adoption.',
 'NVDA': 'NVIDIA boasts strong financials, driven by its dominance in the GPU market for gaming, AI, and data centers.  High revenue growth and profitability are key strengths.',
 'ORCL': 'Oracle financials are in transition, shifting towards cloud-based services. Revenue growth is moderate, and profitability remains stable.  Legacy businesses still contribute significantly.',
 'QCOM': 'QUALCOMM financials show strong performance driven by its leadership in mobile chipsets and licensing.  Profitability is high, and growth is tied to the mobile market and 5G adoption.',
 'SAP': 'SAP demonstrates steady financials with its enterprise software solutions.  Transition to the cloud is ongoing and impacting revenue growth and profitability.',
 'SMSN': 'Samsung financials are diverse, reflecting its presence in various sectors including mobile phones, consumer electronics, and semiconductors. Profitability varies across divisions but the company holds significant cash reserves.',
 'TCEHY': 'Tencent financials are driven by its dominant position in the Chinese gaming and social media market. Revenue growth is strong but regulatory risks in China impact its performance.',
 'TSLA': 'Tesla financials show strong revenue growth driven by electric vehicle demand, but profitability remains volatile due to production and investment costs. The company high valuation reflects market optimism for future growth.',
 'TSM': 'TSMC, a dominant player in semiconductor manufacturing, showcases robust financials fueled by high demand for its advanced chips. Profitability is strong and the company enjoys a technologically advanced position.',
 }
  return company_overviews.get(content['ticker'], 'No company overwiew found')

def get_stock_price_api(content: Dict[str, Any]) -> str:
  "A function to simulate an API call to collect most recent stock price for a given company."
  stock_prices = {
      'AAPL': 225,
      'ADBE': 503,
      'AMD': 134,
      'AMZN': 202,
      'ASML': 658,
      'AVGO': 164,
      'BABA': 88,
      'BKNG': 4000,
      'CRM': 325,
      'CSCO': 57,
      'GOOGL': 173,
      'IBM': 201,
      'INTU': 607,
      'META': 553,
      'MSFT': 415,
      'NFLX': 823,
      'NOW': 1000,
      'NVDA': 141,
      'ORCL': 183,
      'QCOM': 160,
      'SAP': 228,
      'SMSN': 38,
      'TCEHY': 51,
      'TSLA': 302,
      'TSM': 186}
  return stock_prices.get(content['ticker'], 'No stock price found')

def get_company_news_api(content: Dict[str, Any]) -> str:
  "A function to simulate an API call to collect recent news for a given company."
  news_data = {
      'AAPL': 'Apple unveils new iPhone, market reaction muted amid concerns about slowing growth.',
      'ADBE': 'Adobe integrates AI features into Creative Suite, attracting creative professionals.',
      'AMD': 'AMD gains market share in server CPUs, competing with Intel.',
      'AMZN': 'Amazon stock dips after reporting lower-than-expected Q3 profits due to increased shipping costs.',
      'ASML': 'ASML benefits from high demand for advanced chip manufacturing equipment.',
      'AVGO': 'Broadcom announces new acquisition in the semiconductor space.',
      'BABA': 'Alibaba stock faces uncertainty amid ongoing regulatory scrutiny in China.',
      'BKNG': 'Booking Holdings stock recovers as travel demand rebounds post-pandemic.',
      'CRM': 'Salesforce launches new AI-powered CRM tools for enterprise customers.',
      'CSCO': 'Cisco stock rises after positive earnings report, focus on networking solutions.',
      'GOOGL': 'Alphabet announces new AI-powered search features, aiming to compete with Microsoft.',
      'IBM': 'IBM focuses on hybrid cloud solutions, showing steady growth in enterprise segment.',
      'INTU': 'Intuit stock dips after announcing price increases for its tax software.',
      'META': 'Meta shares rise after positive user growth figures in emerging markets.',
      'MSFT': 'Microsoft expands AI integration across its product suite, boosting investor confidence.',
      'NFLX': 'Netflix subscriber growth slows, competition heats up in streaming landscape.',
      'NOW': 'ServiceNow sees strong growth in its cloud-based workflow automation platform.',
      'NVDA': 'Nvidia stock jumps on strong earnings forecast, driven by AI demand.',
      'ORCL': 'Oracle cloud revenue continues strong growth, exceeding market expectations.',
      'QCOM': 'Qualcomm expands its 5G modem business, partnering with major smartphone manufacturers.',
      'SAP': 'SAP cloud transition continues, but faces challenges in attracting new clients.',
      'SMSN': 'Samsung unveils new foldable phones, looking to gain market share.',
      'TCEHY': 'Tencent faces regulatory pressure in China, impacting investor sentiment.',
      'TSLA': 'Tesla stock volatile after price cuts and production increases announced.',
      'TSM': 'TSMC reports record chip demand but warns of potential supply chain disruptions.'
  }
  return news_data.get(content['ticker'], 'No news available')

def get_company_sentiment_api(content: Dict[str, Any]) -> str:
  "A function to simulate an API call to collect market company sentiment for a given company."

  company_sentiment = {'AAPL': 'Neutral',
        'ADBE': 'Neutral',
        'AMD': 'Neutral',
        'AMZN': 'Neutral',
        'ASML': 'Bearish/Undervalued',
        'AVGO': 'Neutral',
        'BABA': 'Neutral',
        'BKNG': 'Neutral',
        'CRM': 'Neutral',
        'CSCO': 'Neutral',
        'GOOGL': 'Neutral',
        'IBM': 'Neutral',
        'INTU': 'Mixed/Bullish',
        'META': 'Neutral',
        'MSFT': 'Neutral',
        'NFLX': 'Neutral',
        'NOW': 'Bullish/Overvalued',
        'NVDA': 'Neutral',
        'ORCL': 'Neutral',
        'QCOM': 'Neutral',
        'SAP': 'Neutral',
        'SMSN': 'Neutral',
        'TCEHY': 'Neutral',
        'TSLA': 'Slightly Overvalued',
        'TSM': 'Neutral',
        }
  return company_sentiment.get(content['ticker'], 'No sentiment available')

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## III. Automated prompt design with Vertex AI prompt optimizer

### Load the dataset

Load the AI financial assistant's question-answer dataset from a Google Cloud Storage bucket. The dataset contains the following columns:

* **question:** User's query regarding about company.
* **tool_names:** Specifies tool names used to answer the question.
* **tool_call:** Details the input parameters passed to the specified tools.
* **tool_call_response:** Raw output from the tool.
* **answer:**  A refined and human-readable response grounded in the tool's output, answering the user's question.

In [ ]:
prompt_optimization_df = pd.read_json(INPUT_DATA_FILE_URI, lines=True)

In [ ]:
prompt_optimization_df.head()

Print an example of the cooking question-answer dataset.  

In [ ]:
vapo_lib.print_df_rows(prompt_optimization_df, n=1)

### Optimize the prompt template with Vertex AI prompt optimizer with custom metric


#### Prepare the prompt template you want to optimize

A prompt consists of two key parts:

* **System Instruction Template** which is a fixed part of the prompt that control or alter the model's behavior across all queries for a given task.

* **Prompt Template** which is a dynamic part of the prompt that changes based on the task. Prompt template includes examples, context, task and more. To learn more, see [components of a prompt](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-design-strategies#components-of-a-prompt) in the official documentation.

In this scenario, you use Vertex AI prompt optimizer to optimize a simple system instruction template.

And you use some examples in the remaining prompt template for evaluating different instruction templates along the optimization process.

> **Note**: Having the `target` placeholder in the prompt template is optional. It represents the prompt's ground truth response in your prompt optimization dataset that you aim to optimize for your templates. If you don't have the prompt's ground truth response, remember to set the `source_model` parameter to your prompt optimizer configuration (see below) instead of adding ground truth responses. Vertex AI prompt optimizer would run your sample prompts on the source model to generate the ground truth responses for you.

In [ ]:
SYSTEM_INSTRUCTION_TEMPLATE = """
Answer the question using correct tools.
"""

PROMPT_TEMPLATE = """
Some examples of correct tools associated to a question are:
Question: {question}
Target tools: {target}
"""

#### Prepare the prompt optimization dataset

To use Vertex AI prompt optimizer, you'll need a CSV or JSONL file with labeled examples.  These examples should follow a specific naming convention. For details see [Optimize prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer).


> **Note**: For effective **prompt optimization**, provide a dataset of examples where your model is poor in performance when using current system instruction template. For reliable results, use 50-100 distinct samples.

> In case of **prompt migration**, consider using the source model to label examples that the target model struggles with, helping to identify areas for improvement.

In [ ]:
train_prompt_optimization_df, test_prompt_optimization_df = train_test_split(
    prompt_optimization_df, test_size=0.3, random_state=8
)

In [ ]:
prepared_train_prompt_optimization_df = train_prompt_optimization_df.copy()

# Prepare target column
prepared_train_prompt_optimization_df['target'] = prepared_train_prompt_optimization_df.apply(vapo_lib.create_target_column, axis=1)

# Remove uneccessary columns
prepared_train_prompt_optimization_df = prepared_train_prompt_optimization_df.drop(
    columns=["tool_names", "tool_arguments", "tool_call_response", "answer"]
)

Print some examples of the prompt optimization dataset.  

In [ ]:
prepared_train_prompt_optimization_df.head()

#### Upload samples to bucket

Once you prepare your prompt optimization dataset, you can upload them on Cloud Storage bucket.

In [ ]:
prepared_train_prompt_optimization_df.to_json(
    INPUT_OPTIMIZATION_DATA_FILE_URI, orient="records", lines=True
)

#### Configure tool settings and validate them

To optimize prompts for using external tools with the Vertex AI SDK, define the tools' functionalities using the `FunctionDeclaration` class. This class uses an OpenAPI-compatible schema to structure the tool definitions.  Your system prompt should be designed to effectively leverage these defined functions.  See the [Introduction to function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more information.  

Example function definitions for a financial assistant are provided below.




In [ ]:
get_company_information = FunctionDeclaration(
    name="get_company_information",
    description="Retrieves financial performance to provide an overview for a company.",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker for a given company",
            }
        },
        "required": [
                    "ticker"
                ]
    },
)

get_stock_price = FunctionDeclaration(
    name="get_stock_price",
    description="Only returns the current stock price (in dollars) for a company.",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "integer",
                "description": "Stock ticker for a company",
            }
        },
        "required": [
                    "ticker"
                ]
    },
)

get_company_news = FunctionDeclaration(
    name="get_company_news",
    description="Get the latest news headlines for a given company.",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker for a company.",
            }
        },
        "required": [
                    "ticker"
                ]
    },
)

get_company_sentiment = FunctionDeclaration(
    name="get_company_sentiment",
    description="Returns the overall market sentiment for a company.",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker for a company",
            },
        },
        "required": [
                    "ticker"
                ]
    },
)

After implementing your functions, wrap each one as a `Tool` object. This allows the Gemini model to discover and execute these functions.  `ToolConfig` provides additional parameters to control how the model interacts with the tools and chooses which function to call.  

Further information can be found in the [Introduction to function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling).



In [ ]:
tools = Tool(function_declarations=[
        get_company_information,
        get_stock_price,
        get_company_news,
        get_company_sentiment,
    ])

tool_config = ToolConfig(
        function_calling_config=ToolConfig.FunctionCallingConfig(
            mode=ToolConfig.FunctionCallingConfig.Mode.ANY,
            allowed_function_names=["get_company_information", "get_stock_price", "get_company_news", "get_company_sentiment"],
        )
    )

To use Vertex AI Prompt Optimizer for tool calling optimization, provide `FunctionDeclaration` and `ToolConfig` as JSON structures (see example below). Vertex AI Prompt Optimizer uses those structures along the optimization process.

Tool Calls json:
```
{"tools": [{"function_declarations": [{"name": "function_1", "description": "My function 1", "parameters": {"type": "OBJECT", "properties": {"argument_1": {"type": "STRING", "description": "My argument 1"}}, "required": ["argument_1"], "property_ordering": ["argument_1"]}}, ...]}]}
```
Function Calling Configuration json:
```
{"function_calling_config": {"mode": "your_mode", "allowed_function_names": ["tool_name_1", ...]}}
```

Below you have some helper functions to get those structures and validate them.



In [ ]:
vapo_tools = json.dumps(
    {
    "tools": [vapo_lib.replace_type_key(tools.to_dict())]
    }
)

vapo_tool_config = json.dumps(vapo_lib.tool_config_to_dict(tool_config))

vapo_lib.validate_tools(vapo_tools)
vapo_lib.validate_tool_config(vapo_tool_config)

#### Configure optimization settings

Vertex AI prompt optimizer lets you control the optimization process by specifying what to optimize (instructions only, demonstrations only, or both), providing a system instruction and prompt template, and selecting the target model.  You can optionally refine the optimization with some advanced settings like its duration and the number of optimization iterations it runs, which models the Vertex AI prompt optimizer uses, and other parameters to control the structure and content of prompts. Below you have some common and recommended default configurations.

In this scenario, you set two additional parameters:

* `tools` parameter to pass tool definitions

* `tool_config` parameter to pass tool configuration

For more advanced control, you can learn and explore more about all the parameters and how to best use them in the [detailed documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer).


In [ ]:
PROMPT_OPTIMIZATION_JOB = "auto-prompt-design-job-" + vapo_lib.get_id()
OUTPUT_OPTIMIZATION_RUN_URI = str(
    OUTPUT_OPTIMIZATION_DATA_URI / PROMPT_OPTIMIZATION_JOB
)

args = Namespace(
    # Basic configuration
    system_instruction=SYSTEM_INSTRUCTION_TEMPLATE,  # System instructions for the target model. String.
    prompt_template=PROMPT_TEMPLATE,  # Template for prompts,  String.
    target_model="gemini-1.5-flash-001",  # Target model for optimization. String. Supported models: "gemini-1.5-flash-002", "gemini-1.5-pro-002", "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-pro-001", "gemini-1.0-pro-002", "gemini-1.0-ultra-001", "text-bison@001", "text-bison@002", "text-bison32k@002", "text-unicorn@001"
    optimization_mode="instruction",  # Optimization mode. String. Supported modes: "instruction", "demonstration", "instruction_and_demo"
    tools=vapo_tools,
    tool_config=vapo_tool_config,
    eval_metrics_types=[
        "tool_name_match",
        "tool_parameter_key_match",
        "tool_parameter_kv_match",
    ],  # List of evaluation metrics. List of strings. Supported metrics: "bleu", "coherence", "exact_match", "fluidity", "fulfillment", "groundedness", "rouge_1", "rouge_2", "rouge_l", "rouge_l_sum", "safety", "question_answering_correctness", "question_answering_helpfulness", "question_answering_quality", "question_answering_relevance", "summarization_helpfulness", "summarization_quality", "summarization_verbosity", "tool_name_match", "tool_parameter_key_match", "tool_parameter_kv_match"
    eval_metrics_weights=[
        0.4, 0.3, 0.3
    ],  # Weights for evaluation metrics. List of floats.  Length must match eval_metrics_types.  Should sum to 1.
    aggregation_type="weighted_sum",  # Aggregation type for evaluation metrics. String. Supported aggregation types: "weighted_sum", "weighted_average"
    input_data_path=INPUT_OPTIMIZATION_DATA_FILE_URI,  # Cloud Storage URI to input optimization data. String.
    output_path=OUTPUT_OPTIMIZATION_RUN_URI,  # Cloud Storage URI to save optimization results. String.
    project=PROJECT_ID,  # Google Cloud project ID. String.

    # (Optional) Advanced configuration
    num_steps=10,  # Number of iterations in instruction optimization mode. Integer between 10 and 20.
    num_template_eval_per_step=2,  # Number of system instructions generated and evaluated in instruction and instruction_and_demo mode. Integer between 1 and 4.
    num_demo_set_candidates=10,  # Number of demonstrations evaluated in instruction and instruction_and_demo mode. Integer between 10 and 30.
    demo_set_size=3,  # Number of demonstrations generated per prompt. Integer between 3 and 6.
    target_model_location="us-central1",  # Location of the target model. String. Default us-central1.
    optimizer_model="gemini-1.5-pro-001",  # Optimization model. String. Supported models: "gemini-1.5-flash-002", "gemini-1.5-pro-002", "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-pro-001", "gemini-1.0-pro-002", "gemini-1.0-ultra-001", "text-bison@001", "text-bison@002", "text-bison32k@002", "text-unicorn@001"
    optimizer_model_location="us-central1",  # Location of the optimization model. String. Default us-central1.
    eval_model="gemini-1.5-pro-001",  # Evaluation model. String. Supported models: "gemini-1.5-flash-002", "gemini-1.5-pro-002", "gemini-1.5-flash-001", "gemini-1.5-pro-001", "gemini-1.0-pro-001", "gemini-1.0-pro-002", "gemini-1.0-ultra-001", "text-bison@001", "text-bison@002", "text-bison32k@002", "text-unicorn@001"
    eval_model_location="us-central1",  # Location of the evaluation model. String. Default us-central1.
    source_model="",  # Google model that the system instructions and prompts were previously used with. String. Not needed if you provide target column.
    source_model_location="",  # Location of the source model. String. Default us-central1. Not needed if you provide target column.
    target_model_qps=1,  # The queries per second (QPS) sent to the target model. Integer greater or equal than 1 depending on your quota.
    optimizer_model_qps=1,  # The queries per second (QPS) sent to the optimization model. Integer greater or equal than 1 depending on your quota.
    eval_qps=1,  # The queries per second (QPS) sent to the eval model. Integer greater or equal than 1 depending on your quota.
    source_model_qps="",  # The queries per second (QPS) sent to the source model. Integer greater or equal than 1 depending on your quota.
    response_mime_type="text/plain",  # MIME response type that the target model uses. String. Supported response: text/plain, application/json.
    response_schema="",  # Response schema that the target model uses to generate answers. String.
    language="English",  # Language of the system instructions. String. Supported languages: "English", "French", "German", "Hebrew", "Hindi", "Japanese", "Korean", "Portuguese", "Simplified Chinese", "Spanish", "Traditional Chinese"
    placeholder_to_content=json.loads(
        "{}"
    ),  # Placeholder to replace any parameter in the system instruction. Dict.
    data_limit=50,  # Amount of data used for validation. Integer between 5 and 100.
)

#### Upload Vertex AI prompt optimizer Cloud Storage

After you define Vertex AI prompt optimizer configuration, you upload them on Cloud Storage bucket.


In [ ]:
args = vars(args)

with epath.Path(CONFIG_FILE_URI).open("w") as config_file:
    json.dump(args, config_file)
config_file.close()

#### Run the automatic prompt optimization job

Now you are ready to run your first Vertex AI prompt optimizer job using the Vertex AI SDK for Python.

> This prompt optimization job requires ~ 40 minutes to run.

> Be sure you have provisioned enough queries per minute (QPM) quota implementing the recommended QPM for each model. If you configure the Vertex AI prompt optimizer with a QPM that is higher than the QPM than you have access to, the job might fail. [Check out](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-optimizer#before-you-begin) the documentation to know more.


In [ ]:
WORKER_POOL_SPECS = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": APD_CONTAINER_URI,
            "args": ["--config=" + CONFIG_FILE_URI],
        },
    }
]

custom_job = aiplatform.CustomJob(
    display_name=PROMPT_OPTIMIZATION_JOB,
    worker_pool_specs=WORKER_POOL_SPECS,
)

custom_job.submit(service_account=SERVICE_ACCOUNT)

### Collect and display the optimization results

Vertex AI prompt optimizer returns both optimized templates and evaluation results for either instruction, or demostrations, or both depending on the optimization mode you define as JSONL files on Cloud Storage bucket. Those results help you understand the optimization process.

In this case, you want to collect the optimized templates and evaluation results for the system instruction.

Below you use a helper function to display those results.

In [ ]:
results_ui = vapo_lib.ResultsUI(OUTPUT_OPTIMIZATION_RUN_URI)
results_df_html = """

"""

display(HTML(results_df_html))
display(results_ui.get_container())

### Evaluate the quality of generated responses with the optimized instruction

Finally, you evaluate generated responses with the optimized instruction qualitatively.

If you want to know how to evaluate the new generated responses quantitatively, check out the [`vertex_ai_prompt_optimizer_sdk` notebook](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/prompts/prompt_optimizer) in the official repository.


#### Generate new responses using the optimized system instruction.

Set the optimized system instruction template you get from Vertex AI prompt optimizer job.

In [ ]:
OPTIMIZED_SYSTEM_INSTRUCTION_TEMPLATE = "To provide the most accurate response to the given question, determine and employ the most suitable tools." # @param {type:"string"}

Prepare optimized prompts using the optimized system instruction template.

In [ ]:
OPTIMIZED_PROMPT_TEMPLATE = (
    OPTIMIZED_SYSTEM_INSTRUCTION_TEMPLATE
    + "\nQuestion: \n{question}"
    + "\nAnswer:"
)

optimized_prompts = [
    OPTIMIZED_PROMPT_TEMPLATE.format(question=q)
    for q in zip(
        test_prompt_optimization_df["question"].to_list(),
    )
]

Leverage Gemini API on Vertex AI to send parallel generation requests.

In this scenario, you have to define a function map which allows you to handle function calling in parallel. 

In [ ]:
function_map = {
    "get_company_information": get_company_information_api,
    "get_stock_price": get_stock_price_api,
    "get_company_news": get_company_news_api,
    "get_company_sentiment": get_company_sentiment_api,
  }

In [ ]:
gemini_llm = vapo_lib.init_new_model(model_name="gemini-1.5-flash-001")

gemini_predictions = [
    vapo_lib.async_generate(p, gemini_llm, function_map, tools, tool_config) for p in optimized_prompts
]

gemini_predictions_col = await tqdm_asyncio.gather(*gemini_predictions)

#### Evaluate new responses

Prepare the test dataset and inspect new responses.

In [ ]:
test_prompt_optimization_df["optimized_prompt_with_vapo"] = optimized_prompts
test_prompt_optimization_df["gemini_answer_with_vapo"] = gemini_predictions_col

In [ ]:
vapo_lib.print_df_rows(test_prompt_optimization_df, n=1)

## IV. Clean up

In [ ]:
delete_bucket = False
delete_job = False
delete_tutorial = False

if delete_bucket:
    ! gsutil rm -r {BUCKET_URI}

if delete_job:
    custom_job.delete()

if delete_tutorial:
    import shutil

    shutil.rmtree(str(TUTORIAL_PATH))